In [1]:
#Pobieranie potrzebnych bibliotek
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
#Ładowanie danych i sprawdzenie kolumn
data = pd.read_excel('./artifacts/Mincer.xlsx')
print(data.columns)

data = data.replace({True: 1, False: 0})
print(data.head(1))

Index(['Unnamed: 0', 'doswiadczenie', 'pensja', 'najwyzsze', 'matura',
       'doswiadczenie_log', 'czy_ma_mature', 'czy_ma_lic', 'czy_ma_inz',
       'czy_ma_mgr', 'czy_ma_mgrinz', 'czy_ma_podyplomowe', 'czy_ma_mba',
       'czy_ma_phd', 'kujawsko_pomorskie', 'lubelskie', 'lubuskie', 'lodzkie',
       'malopolskie', 'mazowieckie', 'opolskie', 'podkarpackie', 'podlaskie',
       'pomorskie', 'slaskie', 'swietokrzyskie', 'warminsko_mazurskie',
       'wielkopolskie', 'zachodniopomorskie', 'male_miasto', 'srednie_miasto',
       'duze_miasto', 'kobieta', 'ln_wyn', 'edukacja_stp'],
      dtype='object')
   Unnamed: 0  doswiadczenie  pensja  najwyzsze  matura  doswiadczenie_log  \
0           0             39    2500        NaN       0           3.688879   

   czy_ma_mature  czy_ma_lic  czy_ma_inz  czy_ma_mgr  ...  swietokrzyskie  \
0              0           0           0           0  ...               0   

   warminsko_mazurskie  wielkopolskie  zachodniopomorskie  male_miasto  \
0     

In [3]:
# Deklarowanie zmiennych potrzebnych do stworzenia modelu
zmienne_niezalezne = [
    'doswiadczenie', 
    'kobieta', 
    'czy_ma_mature', 
    'czy_ma_lic', 
    'czy_ma_inz',
    'czy_ma_mgr', 
    'czy_ma_mgrinz', 
    'czy_ma_podyplomowe', 
    'czy_ma_mba',
    'czy_ma_phd'
]

y = np.log(data['pensja'])

kwantyle = [0.25, 0.50, 0.75]

wyniki = []

In [4]:
# Model
for q in kwantyle:
    formula = f'y ~ ' + ' + '.join(zmienne_niezalezne)
    model = smf.quantreg(formula, data=data)
    quantreg_results = model.fit(q=q)
    wyniki.append(quantreg_results)

print(wyniki)

[<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001EC66993650>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001EC7FE26450>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001EC7FDF4750>]


In [5]:
# Zapisanie wyników
for i, result in enumerate(wyniki):
    with open(f'./wyniki_model4_minc_kwant/eduakcja_zakodowana/bez_dosw_sqrt/kwantyl_{kwantyle[i]}_wyniki.txt', 'w') as file:
        file.write(result.summary().as_text())

In [6]:
# Wyświetlenie estymatorów do interpretacji
decomposition_results = {}

for i, result in enumerate(wyniki):
    quantile = kwantyle[i]  
    decomposition_results[quantile] = {}
    
    for var in zmienne_niezalezne:
        effect = result.params[var]
        decomposition_results[quantile][var] = effect

for quantile, effects in decomposition_results.items():
    print(f"Quantile {quantile}:")
    for var, effect in effects.items():
        print(f"{var}: {effect:.4f}")

Quantile 0.25:
doswiadczenie: 0.0021
kobieta: -0.3071
czy_ma_mature: 0.1714
czy_ma_lic: 0.2263
czy_ma_inz: 0.2529
czy_ma_mgr: 0.3459
czy_ma_mgrinz: 0.4161
czy_ma_podyplomowe: 0.4128
czy_ma_mba: 1.6388
czy_ma_phd: 0.4246
Quantile 0.5:
doswiadczenie: 0.0035
kobieta: -0.3224
czy_ma_mature: 0.1707
czy_ma_lic: 0.2796
czy_ma_inz: 0.4505
czy_ma_mgr: 0.4057
czy_ma_mgrinz: 0.6464
czy_ma_podyplomowe: 0.4125
czy_ma_mba: 1.6180
czy_ma_phd: 0.5067
Quantile 0.75:
doswiadczenie: 0.0043
kobieta: -0.3699
czy_ma_mature: 0.2018
czy_ma_lic: 0.2802
czy_ma_inz: 0.5644
czy_ma_mgr: 0.4682
czy_ma_mgrinz: 0.7148
czy_ma_podyplomowe: 0.4509
czy_ma_mba: 1.5099
czy_ma_phd: 0.4942


In [7]:
# VIF
new_df = data[zmienne_niezalezne].copy()

vif = pd.DataFrame()
vif["Variable"] = new_df.columns
vif["VIF"] = [variance_inflation_factor(new_df.values, i) for i in range(new_df.shape[1])]
print(vif)

             Variable       VIF
0       doswiadczenie  1.726378
1             kobieta  1.988497
2       czy_ma_mature  1.368604
3          czy_ma_lic  1.112129
4          czy_ma_inz  1.017536
5          czy_ma_mgr  1.421955
6       czy_ma_mgrinz  1.054514
7  czy_ma_podyplomowe  1.153283
8          czy_ma_mba  1.003679
9          czy_ma_phd  1.001957
